## Trabalho #2 - Descritores de Características

#### Integrantes:
- Filipe Rodrigues Batista de Oliveira - 2018055091;
- Hilário Côrrea da Silva Neto - 2019006701; 

In [1]:
import os
import cv2
import sklearn
import numpy as np
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE 

HOG:

1. Calcule o gradiente da imagem;
2. Subdivida o resultado de (1) em w x h regiões;
3. Para cada região gere o histograma (vetor de orientações do gradiente X intensidade do gradiente) de (2)
4. Normalize o histograma obtido em (3);
5. Concatene os histogramas de todas as regiões;

In [2]:
def HOGImage(image, w, h, s):
    
    # Calcula o gradiente da imagem, e em seguida sua magnitude e orientação.
    grad_image = (cv2.Sobel(image, cv2.CV_32F, 1, 0, ksize=1), cv2.Sobel(image, cv2.CV_32F, 0, 1, ksize=1))
    mag_grad_image, ang_grad_image = cv2.cartToPolar(grad_image[0], grad_image[1], angleInDegrees=True)
    
    # Percorre toda a imagem (com deslocamento s) gerando histogramas para cada sub-região w x h.
    for i in range(0, w + 1, s):
        features = []
        for j in range(0, h + 1, s):
            # print(mag_grad_image[i:w+i, j:h +j])
            # print("Ang", ang_grad_image[i:w+i, j:h +j])
            
            # NOTA: Perceba que os ângulos dos gradientes está no intervalo de [0º, 180º] após dividir o valor original por 2.
            hist_k = gerar_histograma(mag_grad_image[i:w + i, j:h + j], ang_grad_image[i:w + i, j:h + j] / 2)
            # print(hist_k)
            
        # Concatena os histogramas (normalizados) das sub-regiões w x h. 
        features = np.hstack((features, hist_k / np.linalg.norm(hist_k)))
    
    return features.reshape(1, -1)

In [3]:
def gerar_histograma(mag_image, ang_image):
    
    # 9 bins com diferença de 20º cada.
    bins = np.arange(0, 180, 20)
    hist = dict.fromkeys(bins, 0)
    
    for ang, mag in zip(np.ravel(ang_image), np.ravel(mag_image)):
        # Encontra em qual intervalo (bins) que a orientação do gradiente (ang) está.
        i = np.digitize(ang, bins, right=True)
        
        # Se ângulo for maior que 160ª. 
        if ang > bins[-1]:
            hist[bins[-1]] += mag / 2
            hist[bins[0]] += mag / 2
        elif ang == bins[i]:
            hist[bins[i]] += mag / 2
            hist[bins[i - 1]] += mag / 2
        else:
            hist[bins[i]] += mag
            
    return np.array(list(hist.items()))[:, 1]

Inicialmente vamos definir as classes de imagens que devem ser lidas.

In [4]:
DIR_RAIZ = "fashion_mnist_sample/"
classes = ["Pullover", "Sandal", "Trouser"]

dir_classes = [os.path.join(DIR_RAIZ, classe) for classe in classes]  
caminho2imgs = [[os.path.join(d, f) for f in os.listdir(d) if f.endswith(".png")] for d in dir_classes]

Agora, vamos passar as imagens de cada uma das classes definidas acima como argumento no algoritmo implementado anteriormente.

In [11]:
imgs = [cv2.imread(caminho2img, cv2.IMREAD_GRAYSCALE) for caminho2img in caminho2imgs[0]]
# HOGImage(imgs[0], 16,16,8)
descritores3classes = np.array([HOGImage(img, 16, 16, 8) for img in imgs]).reshape(-1, 9)
# descritores3classes

Veja que a utilização do HOG permitiu que nós conseguissemos separar as classes após usarmos o TSNE

In [34]:
pca = PCA(n_components=9)
tsne = TSNE(random_state=0)

resultado_pca = pca.fit_transform(descritores3classes)
resultado_tsne = tsne.fit_transform(descritores3classes)

fig = px.scatter(
    resultado_tsne, x=0, y=1,
    color=classes , labels={"color": "classes"}
)

fig.show()

/usr/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



ValueError: All arguments should have the same length. The length of argument `color` is 3, whereas the length of  previously-processed arguments ['index', '0', '1'] is 100

Repetindo a análise precendente, só que agora usando todas as classes no banco de dados. </br>
Perceba que 

In [ ]:
classes = os.listdir(DIR_RAIZ)

dir_classes = [os.path.join(DIR_RAIZ, classe) for classe in classes]  
caminho2imgs = [[os.path.join(d, f) for f in os.listdir(d) if f.endswith(".png")] for d in dir_classes]

imgs = [cv2.imread(caminho2img, cv2.IMREAD_GRAYSCALE) for caminho2img in caminho2imgs[0]]
descritores = [HOGImage(img, 16, 16, 8) for img in imgs] 

pca = PCA(n_components=9) 
tsne = TSNE(random_state=0)

resultado_pca = pca.fit_transform(descritor)
resultado_tsne = tsne.fit_transform(resultado_pca)

# fig = px.scatter(
#     resultado_tsne, x=0, y=1,
#     color=np.arange(resultado_tsne.shape[0]) , labels={"color": "classes"}
# )

# fig.show()

Ao se utilizar o descritor SIFT vemos que 

In [ ]:
sift = cv2.SIFT.create()
_, descritor2 = sift.detectAndCompute(imgs, None)

pca = PCA(n_components=9) 
tsne = TSNE(random_state=0)

resultado_pca = pca.fit_transform(descritor2)
resultado_tsne = tsne.fit_transform(resultado_pca)

fig = px.scatter(
    resultado_tsne, x=0, y=1,
    color=descritor2, labels={"color": "classes"}
)

fig.show()